In [15]:
import pandas as pd
import time
import sys
from IPython.display import clear_output
from datetime import date, datetime


In [16]:
def SaveRequest(feature:str, stdID:int):
    with open(f"std{stdID}PreviousRequests.txt", "a") as f:
        date_now = date.today().strftime("%d/%m/%Y")
        time_now = datetime.now().strftime("%H:%M %p")
        text =f"\n{feature}\t\t{date_now}\t\t{time_now}\t" 
        f.write(text)
        f.close()

In [17]:
def menuFeature(stdID:int, degrees: list):
    print(
        """Student Transcript Generation System
        ===================================================
        1. Student details
        2. Statistics
        3. Transcript based on major courses
        4. Transcript based on minor courses
        5. Full transcript
        6. Previous transcript requests
        7. Select Another student
        8. Terminate the system
        ===================================================
        """
        )
    # this is to count feature requests
    # each request we will add one 
    feature_requested = 0
    while True:
        # while true this condition active before user dont accept correct values for ex. 1-8
        # try and except tru catch error , this users gives to enter only integer values.
        try:
            feature = int(input("Enter Your Feature: "))
            if feature <1 and feature >8: 
                raise ValueError 
        except ValueError:
            print("Please Enter Correct Value")
        if feature == 1:
            SaveRequest("Details", stdID)
            feature_requested+=1 #adding one to count how many request sent
            detailsFeature(stdID, degrees)
        elif feature == 2:
            SaveRequest("Statistics", stdID)
            feature_requested+=1 #adding one to count how many request sent
            statisticsFeature(stdID, degrees)
        elif feature == 3:
            SaveRequest("Major", stdID)
            feature_requested+=1 #adding one to count how many request sent
            majorTranscriptFeature(stdID, degrees)
        elif feature == 4:
            SaveRequest("Minor", stdID)
            feature_requested+=1 #adding one to count how many request sent
            minorTranscriptFeature(stdID, degrees)
        elif feature == 5:
            SaveRequest("Full", stdID)
            feature_requested+=1 #adding one to count how many request sent
            fullTranscriptFeature(stdID, degrees)
        elif feature == 6:
            feature_requested+=1 #adding one to count how many request sent
            previousRequestsFeature(stdID, degrees)
        elif feature == 7:
            feature_requested+=1 #adding one to count how many request sent
            newStudentFeature()
        elif feature == 8:
            terminateFeature(feature_requested)
            break
            
            

In [18]:
def startFeature():
    while True:
        try:
            stdID = int(input("Enter Student Id,(i.e. 202006000): "))
        except ValueError:
            print("Please enter integer value Student id")
            continue
        
        # excel file to dataframe using pandas
        df = pd.read_excel("studentDetails.xlsx")
        
        # checking if inputed stdID is exist
        result_df = df[df["stdID"] == stdID]
        
        if result_df.empty:
            print("Wrong id, Acceptable student IDs -->")
            # get the unique student id values (rows)
            distinct_rows = df["stdID"].drop_duplicates()
            #printing avialable student ids
            for r in distinct_rows:
                print(r)
            continue
        break
    
    while True:
        print("Available degrees")
        # getting available degrees and printing
        degrees = result_df.Degree
        for degree in degrees:
            print(degree)
        i_degree = input("Please enter degree(s) from available degrees (i.e. BS1 M1): ").upper().split(" ")
        if len(i_degree) < 1:
            print("Please enter degree(s) corectly")
            continue
        # checking if user entered correct  degress 
        # and getting which similar degress from two list
        ds = list(set(i_degree).intersection(degrees))   #find common elements between two lists in Python
        # getting ready header for PreviousRequestFeature.txt file
        # Then we will add for each request feature in this txt fle
        with open(f"std{stdID}PreviousRequests.txt", "w") as f:
            header ="""Request\t\t\tDate\t\tTime
=========================================="""
            f.write(header)
            f.close()
        # system sleep for 4
        time.sleep(4)
        #Calling menu feature function 
        return menuFeature(stdID, ds)
    
        

In [19]:
def detailsFeature(stdID:int, degrees: list):
    # excel file to dataframe using pandas
    df = pd.read_excel("studentDetails.xlsx")
    result_df = df[df["stdID"] == stdID] # getting rows where student id is similar
    figure2 = """"""
    for degree in degrees:
        row = result_df[result_df["Degree"] == degree] # getting row for each degree
        figure2+=f"""\nName: {row.Name.iloc[0]}
        stdID: {row.stdID.iloc[0]}
        Level(s): {row.Level.iloc[0]}
        Number of terms: {row.Terms.iloc[0]}
        College(s): {row.College.iloc[0]}
        Department(s): {row.Department.iloc[0]}
        """
    print(figure2)

    # storing figures2 in txt file 
    with open(f"std{stdID}details.txt", "w") as f:
        f.write(figure2)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)
    

In [20]:
def statisticsFeature(stdID: int, degrees:list):
    
    # excel file to dataframe using pandas
    df = pd.read_excel(f"{stdID}.xlsx")
    
    figure3 =""""""
    
    # creating figure for each degree
    for degree in degrees:
        result_df = df[df["Degree"] == degree] # getting rows where student degree is similar
        
        figure3+=f"""======================================================
        ************      Graduate({degree}) Level     ************
        ======================================================
        Overal Avarage (major and minor) for all terms:{round(result_df.Grade.mean())},
        Average (major and minor) of each term:{round(result_df.Grade.sum()/result_df.creditHours.sum())}
        """ 
        terms = result_df.Term.unique() # getting unique terms
        for i in terms:
            avg = round(result_df[result_df["Term"] == i]["Grade"].mean()) # gettig avarage of each term
            figure3 += f'\n\tTerm {i}: {avg}'
            
        repeated_courses = result_df[result_df.courseName.duplicated()] # getting one row wherw course is repeated
        # then checking if we have repeated row or not
        if not repeated_courses.empty:
            repeated = f"Yes, {repeated_courses['courseName'].iloc[0]}"
        else:
            repeated = "No" 
            
        max_grade = result_df[result_df["Grade"] == result_df["Grade"].max()] #Getting row where is max grade
        min_grade = result_df[result_df["Grade"] == result_df["Grade"].min()] #getting row where is min grade
        #getting term and grade in bellow lines
        figure3+=f"""
        Maximum grade(s) and in which term(s):Term - {max_grade["Term"].iloc[0]}, Grade- {max_grade["Grade"].iloc[0]}. 
        Minimum grade(s) and in which term(s):Term - {min_grade["Term"].iloc[0]}, Grade- {min_grade["Grade"].iloc[0]}.  
        Do you have any repeated course(s)?{repeated},
        """
    
    print(figure3)
    # storing figures3 in txt file 
    with open(f"std{stdID}statistics.txt", "w") as f:
        f.write(figure3)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)
    

In [21]:
def majorTranscriptFeature(stdID: int, degrees:list):
    
    # excel file to dataframe using pandas
    df = pd.read_excel(f"{stdID}.xlsx")
    
    figure4 =""""""
    frames = []
    # We dont know how much degrees will be sleected that's why why are looping in degrees
    # and then joining dataframes together and getting one dataframe 
    # We dont need this to do but is optional we can do with only df at this moment I think
    # its better to do like as i did
    for degree in degrees:
        frames.append(df[df["Degree"] == degree]) 
        
    result_df = pd.concat(frames) # Joining DataFrames 
    #Getting only major courses
    result_df = result_df[result_df["courseType"] == "Major"]
    # need to get student detail info for student id of course.
    df1 = pd.read_excel("studentDetails.xlsx")
    studentdtls_df = df1[df1["stdID"] == stdID] # getting rows where student id is similar
    
    # creating figure for each degree
    for degree in degrees:
        
        detail_df = studentdtls_df[studentdtls_df["Degree"] == degree]
        figure4+=f"""Name: {detail_df.Name.iloc[0]}         stdID: {detail_df.stdID.iloc[0]}
        College: {detail_df.College.iloc[0]}         Department: {detail_df.Department.iloc[0]}
        Major: {detail_df.Major.iloc[0]}         Minor: {detail_df.Minor.iloc[0]}
        Level: {detail_df.Level.iloc[0]}         Number of terms: {detail_df.Terms.iloc[0]}
        """
        
        terms = result_df.Term.unique() # getting unique terms
        for i in terms:
            terms_df = result_df[result_df["Term"]==i] 
            figure4+=f"""======================================================
            ************      Term {i}     ************
            ======================================================
            course id   course name     credit hours    grade""" 
            
            # in one term we can have two courses thats why why are looping
            # and then adding to figure4
            for index, td in terms_df.iterrows():
                c_id = td.courseID
                c_name = td.courseName
                c_hours = td.creditHours
                grade = td.Grade
                figure4+=f"""\n\t\t\t{c_id}    \t\t{c_name}     \t\t\t{c_hours}    \t\t{grade}"""
                
            m_avg = round(terms_df.Grade.sum() / terms_df.creditHours.sum()) # calculate major avg
            o_avg = round(terms_df.Grade.mean()) # calculate overal avg
            figure4+=f"""\n\t\t\tMajor Average = {m_avg}    \t\tOveral Average = {o_avg}
            """
        
        figure4+=f"""======================================================
            ************End of Transcript for Level({detail_df.Level.iloc[0]})  ************
            ======================================================
        """
    print(figure4)
    # storing figures3 in txt file 
    with open(f"std{stdID}MajorTranscript.txt", "w") as f:
        f.write(figure4)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)

In [22]:
def minorTranscriptFeature(stdID: int, degrees:list):
    
    # excel file to dataframe using pandas
    df = pd.read_excel(f"{stdID}.xlsx")
    
    figure4 =""""""
    frames = []
    # We dont know how much degrees will be sleected that's why why are looping in degrees
    # and then joining dataframes together and getting one dataframe 
    # We dont need this to do but is optional we can do with only df at this moment I think
    # its better to do like as i did
    for degree in degrees:
        frames.append(df[df["Degree"] == degree]) 
        
    result_df = pd.concat(frames) # Joining DataFrames 
    #Getting only major courses
    result_df = result_df[result_df["courseType"] == "Minor"]
    # need to get student detail info for student id of course.
    df1 = pd.read_excel("studentDetails.xlsx")
    studentdtls_df = df1[df1["stdID"] == stdID] # getting rows where student id is similar
    
    # creating figure for each degree
    for degree in degrees:
        
        detail_df = studentdtls_df[studentdtls_df["Degree"] == degree]
        figure4+=f"""Name: {detail_df.Name.iloc[0]}         stdID: {detail_df.stdID.iloc[0]}
        College: {detail_df.College.iloc[0]}         Department: {detail_df.Department.iloc[0]}
        Major: {detail_df.Major.iloc[0]}         Minor: {detail_df.Minor.iloc[0]}
        Level: {detail_df.Level.iloc[0]}         Number of terms: {detail_df.Terms.iloc[0]}
        """
        
        terms = result_df.Term.unique() # getting unique terms
        for i in terms:
            terms_df = result_df[result_df["Term"]==i] 
            figure4+=f"""======================================================
            ************      Term {i}     ************
            ======================================================
            course id   course name     credit hours    grade""" 
            
            # in one term we can have two courses thats why why are looping
            # and then adding to figure4
            for index, td in terms_df.iterrows():
                c_id = td.courseID
                c_name = td.courseName
                c_hours = td.creditHours
                grade = td.Grade
                figure4+=f"""\n\t\t\t{c_id}    \t\t{c_name}     \t\t\t{c_hours}    \t\t{grade}"""
                
            m_avg = round(terms_df.Grade.sum() / terms_df.creditHours.sum()) # calculate major avg
            o_avg = round(terms_df.Grade.mean()) # calculate overal avg
            figure4+=f"""\n\t\t\tMinor Average = {m_avg}    \t\tOveral Average = {o_avg}
            """
        
        figure4+=f"""======================================================
            ************End of Transcript for Level({detail_df.Level.iloc[0]})  ************
            ======================================================
        """
    print(figure4)
    # storing figures3 in txt file 
    with open(f"std{stdID}MinorTranscript.txt", "w") as f:
        f.write(figure4)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)

In [23]:
def fullTranscriptFeature(stdID: int, degrees:list):
    
    # excel file to dataframe using pandas
    df = pd.read_excel(f"{stdID}.xlsx")
    
    figure5 =""""""
    frames = []
    # We dont know how much degrees will be sleected that's why why are looping in degrees
    # and then joining dataframes together and getting one dataframe 
    # We dont need this to do but is optional we can do with only df at this moment I think
    # its better to do like as i did
    for degree in degrees:
        frames.append(df[df["Degree"] == degree]) 
        
    joined_dfs = pd.concat(frames) # Joining DataFrames 
    #Getting only major courses
    #@TODO result_df = result_df[result_df["courseType"] == "Minor"]
    # need to get student detail info for student id of course.
    df1 = pd.read_excel("studentDetails.xlsx")
    studentdtls_df = df1[df1["stdID"] == stdID] # getting rows where student id is similar
    
    # creating figure for each degree
    for degree in degrees:
        # Getting rows only for each degree 
        result_df = joined_dfs[joined_dfs["Degree"] == degree] 
        # getting only row where degree is in student details
        detail_df = studentdtls_df[studentdtls_df["Degree"] == degree] 
        
        figure5+=f"""Name: {detail_df.Name.iloc[0]}         stdID: {detail_df.stdID.iloc[0]}
        College: {detail_df.College.iloc[0]}         Department: {detail_df.Department.iloc[0]}
        Major: {detail_df.Major.iloc[0]}         Minor: {detail_df.Minor.iloc[0]}
        Level: {detail_df.Level.iloc[0]}         Number of terms: {detail_df.Terms.iloc[0]}
        """
        
        terms = result_df.Term.unique() # getting unique terms
        # looping in terms(list of term) because we need to add figure for each tearm
        for i in terms:
            terms_df = result_df[result_df["Term"]==i] 
            figure5+=f"""======================================================
            ************      Term {i}     ************
            ======================================================
            course id   course name     credit hours    grade""" 
            
            # in one term we can have two courses thats why why are looping
            # and then adding to figure4
            for index, td in terms_df.iterrows():
                c_id = td.courseID
                c_name = td.courseName
                c_hours = td.creditHours
                grade = td.Grade
                figure5+=f"""\n\t\t\t{c_id}    \t\t{c_name}     \t\t\t{c_hours}    \t\t{grade}"""
                
            # calculating major Average
            major_rows = terms_df[terms_df["courseType"] == "Major"]
            major_avg = round(major_rows.Grade.sum() / major_rows.creditHours.sum()) 
            # calculating minor Average
            minor_rows = terms_df[terms_df["courseType"] == "Minor"]
            minor_avg = round(minor_rows.Grade.sum() / minor_rows.creditHours.sum()) 
            # calculating term Average
            t_avg = round(terms_df.Grade.sum() / len(terms))
            # calculating overal Average
            o_avg = round(terms_df.Grade.mean()) 
            
            figure5+=f"""\n\t\t\tMajor Average = {major_avg}    \tMinor Average = {minor_avg}
            \t\t\tTerm Average = {t_avg}    \t\tOverall Average = {o_avg}
            """
        
        figure5+=f"""======================================================
            ************End of Transcript for Level({detail_df.Level.iloc[0]})  ************
            ======================================================
        """
    print(figure5)
    # storing figures3 in txt file 
    with open(f"std{stdID}FullTranscript.txt", "w") as f:
        f.write(figure5)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)

In [24]:
def previousRequestsFeature(stdID:int, degrees:list):
    # opening
    with open(f"std{stdID}PreviousRequests.txt") as f:
        text = f.read()
        print(text)
        f.close()
        
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    menuFeature(stdID, degrees)

In [25]:
def newStudentFeature():
    time.sleep(5) #sleeping system for 5 seconds
    clear_output(wait=True) #clearing terminal
    startFeature() # calling function to start program for another student

In [26]:
def terminateFeature(count_requests):
    print(f"Program Closed!\n Feature Requested for {count_requests} times")

In [27]:
startFeature()

Available degrees
BS1
M1
D1
Student Transcript Generation System
        1. Student details
        2. Statistics
        3. Transcript based on major courses
        4. Transcript based on minor courses
        5. Full transcript
        6. Previous transcript requests
        7. Select Another student
        8. Terminate the system
        
Program Closed!
 Feature Requested for 0 times
